# Simple interface for helmet on head detection using trained model

## Initialization

#### Import block

In [6]:
# Предварительная установка пакетов (для Colab)
#!pip install torchinfo ultralytics

In [7]:
import numpy as np
import torch
import matplotlib.image as img
import matplotlib.pyplot as plt
import matplotlib
#import cv2
import os
import json
import torchvision.transforms.functional as F

#import typing
#import xml.etree.ElementTree as ET
#import random

from tqdm import tqdm
from PIL import Image
from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image
from torchvision.ops import box_convert
from torchinfo import summary
from torchvision.io.image import read_image
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights, fasterrcnn_resnet50_fpn
from torchvision.models.mobilenetv3 import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image, pil_to_tensor
from torchvision import transforms
from torchvision.transforms._presets import ObjectDetection
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.nn.functional import smooth_l1_loss, cross_entropy
#?
from torchvision.models.detection.ssdlite import SSDLiteClassificationHead
#from ultralytics import YOLO

%matplotlib inline

#### Choose place and set default path to work

In [8]:
# Устанавливаем путь до датасета

# Локально
#ds_path = '/home/vovk/SberUniversity/DS_from_03_10_2022/PM_Group_AN/FinalProj/HelmetDetection/VOC2028/'
# relative path
ds_path = 'VOC2028/'

# Colab
# from google.colab import drive
# drive.mount('/content/drive')

# ds_path = '/content/drive/MyDrive/SberUniversity/PM_DS14AN/Fin/HelmetDetection/VOC2028/'

## Prepairing **SSDLite** model for prediction.

#### Create template for loading model.

In [9]:
model_loaded = ssdlite320_mobilenet_v3_large( weights='DEFAULT',
                                      weights_backbone=MobileNet_V3_Large_Weights.IMAGENET1K_V1,
                                      score_tresh=0.25
                                     )

m_3class = ssdlite320_mobilenet_v3_large( num_classes=3,
                                          weights_backbone=None,
                                          score_tresh=0.25,
                                        )

model_loaded.head = m_3class.head

#### Load model

In [13]:
best_path = ds_path + 'tmp/models/' + 'SSDLiteMobNetFreezBackbone_3class_best.pt'
checkpoint = torch.load(best_path)
model_loaded.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

print('Model trained', epoch, 'epoches')
print(f'Boxes loss: {round(loss["bbox_loss"].item(),4)}, class loss: {round(loss["cls_loss"].item(),4)}')

Model trained 19 epoches
Boxes loss: 1.6028, class loss: 1.9784


# Prediction using trained **SSDLite** model.

In [ ]:
#for Colab
#from google.colab import files
#img1 = files.upload()

In [15]:
img1 = Image.open(ds_path + 'short_test/'+ "hard_hat_workers15.png")
convert_to_tensor = ObjectDetection() # Универсальное преобразование (используется в исходниках ssdlite)
tensor_img1 = convert_to_tensor(img1)
model_loaded.eval()
model_loaded.score_thresh = 0.4
prediction = model_loaded([tensor_img1])[0]

labels_dict={'1': 'head',
             '2': 'helmet'}

labels = [labels_dict[str(label.item())] + ': ' + \
          str(round(prediction["scores"][idx].item(), 2)) \
          for idx, label in enumerate(prediction["labels"])]

box = draw_bounding_boxes(pil_to_tensor(img1), # for original image case
                          #(tensor_img1*256).to(dtype=torch.uint8), # for normalized image case (not fully identical)
                          boxes=prediction['boxes'],
                          labels=labels,
                          colors='red',
                          width=3)
im = to_pil_image(box.detach())
im.show()

print(prediction)

{'boxes': tensor([[359.5716,  57.1137, 447.9247, 186.7525],
        [247.6188,  44.8378, 349.3846, 172.3196],
        [ 19.2173, 212.3813, 113.1847, 309.3752]], grad_fn=<StackBackward0>), 'scores': tensor([0.9081, 0.8517, 0.5148], grad_fn=<IndexBackward0>), 'labels': tensor([2, 2, 2])}
